# Load necessary Python packages

In [1]:
%env R_HOME=/Applications/anaconda/envs/py36andR/lib/R
import rpy2.rinterface
%load_ext rpy2.ipython

import os, re, sys, shutil, fileinput, getopt, argparse, csv, copy
import numpy as np
# import pandas as pd
from itertools import *
from collections import *
from subprocess import * 
from time import *

sys.path.insert(1, '/Users/tuckerely/SUPCRTandEQs/Automate_EQ3')  #    this sets up the environmental variables

import scripts
import scripts.Search_Data0_3    #    this lets the user know what basis and solid options are available.
import scripts.Call_EQ3          #    this builds and runs the EQ3 files
import scripts.EQ3_Error         #    error checking for failed runs.
pwd = os.getcwd()

env: R_HOME=/Applications/anaconda/envs/py36andR/lib/R


# Begin Preprocessing

### Read MasterInput.csv

In [2]:
%%R
# Cells with %%R use R language
# Read the master .csv file input with the filename "MasterInput" by default.
master <- "MasterInput"
f <- read.csv(paste(master, ".csv", sep = ""), header=TRUE, colClasses = "character")
# Convert master file to a matrix
g <- as.matrix(f)
# Check to see if import was successful by showing the first few entries
head(g)

     id          name              year           area            pH      
[1,] "Sample ID" "Sample Location" "Year sampled" "Sampling area" "pH"    
[2,] ""          ""                ""             ""              "Field" 
[3,] "140114U"   "Sun burn"        "2014"         "Falej North"   "11.355"
[4,] "140114U"   "Sun burn"        "2014"         "Falej North"   "11.355"
[5,] "140114V"   "Grey Sauce"      "2014"         "Falej North"   "11.407"
[6,] "140111H"   "Horse Head"      "2014"         "Qafifah"       "10.24" 
     delete       temp_pH   delete.1 delete.2       temp_cond   delete.3
[1,] "pH"         "pH Temp" "Cond."  "Flash. Cond." "Cond Temp" "Time"  
[2,] "millivolts" "oC"      "uS"     "uS"           "oC"        ""      
[3,] "-279.9"     "22.1"    "1479"   ""             "21.7"      "15:10" 
[4,] "-279.9"     "22.1"    "1479"   ""             "21.7"      "15:10" 
[5,] "-285.4"     "24.6"    "1803"   ""             "24.4"      "16:15" 
[6,] "-204.4"     "20.4"    "597"    

### Remove duplicate rows.
If samples with duplicate names are identified, the second sample will not be considered.

In [3]:
%%R
# If there are duplicate sample IDs...
try(
  if (which(duplicated(g[, 1])==TRUE ) > 0 ){
    # ...disregard rows with duplicate sample IDs
    g <- g[-which(duplicated(g[, 1])==TRUE ), ]
  }
)

### Assign sample (row) names
Note that this could not be done until duplicate samples were dealt with in the last step.

In [4]:
%%R
# Assigns rownames to data matrix.
# Note that this could not be done until duplicate sample IDs
# were removed because duplicate rownames are not allowed
# when importing a csv.
rownames(g) <- g[, 1]

# Report number of rows and columns
paste("There are", nrow(g), "rows and", ncol(g), "columns")
head(g)

          id          name              year           area            pH      
Sample ID "Sample ID" "Sample Location" "Year sampled" "Sampling area" "pH"    
          ""          ""                ""             ""              "Field" 
140114U   "140114U"   "Sun burn"        "2014"         "Falej North"   "11.355"
140114V   "140114V"   "Grey Sauce"      "2014"         "Falej North"   "11.407"
140111H   "140111H"   "Horse Head"      "2014"         "Qafifah"       "10.24" 
140111I   "140111I"   "Tadpole Trauma"  "2014"         "Qafifah"       "10.87" 
          delete       temp_pH   delete.1 delete.2       temp_cond   delete.3
Sample ID "pH"         "pH Temp" "Cond."  "Flash. Cond." "Cond Temp" "Time"  
          "millivolts" "oC"      "uS"     "uS"           "oC"        ""      
140114U   "-279.9"     "22.1"    "1479"   ""             "21.7"      "15:10" 
140114V   "-285.4"     "24.6"    "1803"   ""             "24.4"      "16:15" 
140111H   "-204.4"     "20.4"    "597"    ""      

### Assign geochemical variable (column) names and clean up resulting matrix.

In [5]:
%%R

#get original column and row numbers and names
orig_row <- nrow(g)
orig_col <- ncol(g)
orig_rownames <- rownames(g)
orig_colnames <- colnames(g)

#ignoring "id","name","year","area" columns, replace all non-numeric values with NA
g[,seq(5, orig_col)] <- as.numeric(g[, seq(5, orig_col)])

#convert the numeric matrix back to a matrix with row and column names
g <- matrix(g, nrow = orig_row, ncol = orig_col)
rownames(g) <- orig_rownames
colnames(g) <- orig_colnames

### Disregard samples with no pH measurement.

In [6]:
%%R

#delete rows containing NA for pH
g <- g[!is.na(g[, "pH"]), ]

#replace all NA with ""
g[is.na(g)] <- ""

#summarize
head(g)

        id        name             year   area          pH       delete  
140114U "140114U" "Sun burn"       "2014" "Falej North" "11.355" "-279.9"
140114V "140114V" "Grey Sauce"     "2014" "Falej North" "11.407" "-285.4"
140111H "140111H" "Horse Head"     "2014" "Qafifah"     "10.24"  "-204.4"
140111I "140111I" "Tadpole Trauma" "2014" "Qafifah"     "10.87"  "-240.3"
140116C "140116C" "Slippity Slide" "2014" "Shmait"      "8.714"  "-120.1"
140116D "140116D" "Rome"           "2014" "Shmait"      "9.104"  "-144.5"
        temp_pH delete.1 delete.2 temp_cond delete.3 delete.4 delete.5 
140114U "22.1"  "1479"   ""       "21.7"    ""       "608551" "2526487"
140114V "24.6"  "1803"   ""       "24.4"    ""       "608559" "2526485"
140111H "20.4"  "597"    ""       "20.2"    ""       "646074" "2533678"
140111I "19.7"  "683"    ""       "18.8"    ""       "646074" "2533674"
140116C "27.3"  "777"    ""       "27.3"    ""       "486045" "2588468"
140116D "27.4"  "777"    ""       "27.1"    ""    

### Selects measurement to use if multiple measurements are available for a sample, part I
E.g temperature measured by the conductivity meter is favored over that measured by the pH meter. This is represented in the following list by '>', which designates the hierarchy of importance. If the favored measurement isn't available, the next most-important measurement is used instead.
- Temperature. Conductivity meter > pH meter.
- Ammonium, nitrate, nitrite, phosphate. Ion chromatography > field spectrophotometry.
- Iron, potassium, calcium, magnesium, and lithium are handled later.


In [7]:
%%R
#use conductivity meter temperature, or if it is not available,
# instead use pH meter temperature. If neither is available, "NA".

# If temp_cond exists, use it. If not, use temp_pH.
# Replace the column temp_pH with this.
g[seq(1, nrow(g), 1), "temp_pH"] <- ifelse(g[, "temp_cond"] != "", g[, "temp_cond"],
  ifelse(g[, "temp_pH"] != "", g[, "temp_pH"], NA))
#rename the column "temp_pH" with "temp_final" to accurately represent what has been done.
colnames(g)[colnames(g) == "temp_pH"] <- "temp_final"
# Mark "temp_cond" for deletion
colnames(g)[colnames(g) == "temp_cond"] <- "delete"

# Use ion chromatography NH4+ if available, or if not,
# use spectro. If neither is available, use a blank "".
# Both spectro and IC are reported as mg/L and ppm, respectively
# (equivalent units), so columns can be combined into "NH4._final"
# immediately without unit conversion.

# If "NH4." exists, use it. If not, use "NH4._spectro". Replace the column NH4._spectro with this.
g[seq(1, nrow(g), 1), "NH4._spectro"] <- ifelse(g[, "NH4."]!="", g[, "NH4."],
  ifelse(g[, "NH4._spectro"] != "", g[, "NH4._spectro"], ""))
# Rename the column "NH4.spectro" to "NH4._final" to accurately represent what has been done.
colnames(g)[colnames(g)=="NH4._spectro"] <- "NH4._final"
colnames(g)[colnames(g)=="NH4."] <- "delete" # Mark "NH4." for deletion

# Use ion chromatography NO3-, NO2-, PO4-3 if available, or if not,
# use spectro. If neither is available, use a blank "".
# Both spectro and IC are reported as mg/L and ppm, respectively
# (equivalent units), so columns can be combined into "NO3_final"
# "NO2._final", and "PO4.3_final" immediately without unit conversion.

# If "NO3." exists, use it. If not, use "NO3._spectro". Replace the column NO3._spectro with this.
g[seq(1,nrow(g),1),"NO3._spectro"] <- ifelse(g[, "NO3."] != "", g[, "NO3."],
  ifelse(g[, "NO3._spectro"] != "", g[, "NO3._spectro"], ""))
#rename the column "NO3.spectro" to "NO3._final" to accurately represent what has been done.
colnames(g)[colnames(g) == "NO3._spectro"] <- "NO3._final"
colnames(g)[colnames(g) == "NO3."] <- "delete" #mark "NO3." for deletion

#if "NO2." exists, use it. If not, use "NO2._spectro". Replace the column NO2._spectro with this.
g[seq(1, nrow(g), 1), "NO2._spectro"] <- ifelse(g[, "NO2."] != "", g[, "NO2."],
  ifelse(g[, "NO2._spectro"] != "", g[, "NO2._spectro"], ""))
#rename the column "NO2.spectro" to "NO2._final" to accurately represent what has been done.
colnames(g)[colnames(g) == "NO2._spectro"] <- "NO2._final"
colnames(g)[colnames(g) == "NO2."] <- "delete" #mark "NO2." for deletion

#if "PO4.3" exists, use it. If not, use "PO4.3_spectro". Replace the column PO4.3_spectro with this.
g[seq(1, nrow(g), 1), "PO4.3_spectro"] <- ifelse(g[, "PO4.3"] != "", g[, "PO4.3"],
  ifelse(g[, "PO4.3_spectro"] != "", g[, "PO4.3_spectro"], ""))
#rename the column "PO4.3_spectro" to "PO4.3_final" to accurately represent what has been done.
colnames(g)[colnames(g) == "PO4.3_spectro"] <- "PO4.3_final"
colnames(g)[colnames(g) == "PO4.3"] <- "delete" #mark "PO4.3" for deletion

### Delete rows containing NA or blanks for variables crucial for EQ3
# deletes rows with NA for "temp_final" (NAs generated in temp_cond vs temp_pH step above)
g <- g[!is.na(g[, "temp_final"]), ]

#if(which(g[, "Na."] == "") > 0){
#g <- g[-which(g[, "Na."] == ""),] # deletes rows with "" for "Na+" column
#}

#if(which(g[, "Cl."] == "" ) > 0){
#g <- g[-which(g[, "Cl."] == "" ), ] # deletes rows with "" for "Cl-" column
#}

#if(which(g[, "Na."] == 0 ) > 0){
#g <- g[-which(g[, "Na."] == 0 ), ] # deletes rows with 0 for "Na+" column
#}

#if(which(g[, "Cl."] == 0 ) > 0){
#g <- g[-which(g[, "Cl."]==0 ), ] # deletes rows with 0 for "Cl-" column
#}



### Unit conversions

In [8]:
%%R

### DIC unit conversion (DIC ppmc to HCO3- molality)
g[seq(1, nrow(g), 1), "DIC_ppmC"] <- as.numeric(g[seq(1, nrow(g), 1),"DIC_ppmC"])/1000/12.01 #convert ppmC to molality
colnames(g)[colnames(g) == "DIC_ppmC"] <- "HCO3._molality" #rename the column to HCO3- molality

g[is.na(g)] <- "" #replace NAs with 0
g[g == ""] <- 0 #replace "" with 0

#if(which(g[, "HCO3._molality"]== g[, "CaCO3alkalinity"] & g[, "HCO3._molality"] == 0 & g[, "pH"] > 5) > 0){
#g <- g[-which(g[, "HCO3._molality"]== g[, "CaCO3alkalinity"] & g[, "HCO3._molality"] == 0 & g[, "pH"] > 5),] # deletes rows where DIC = CaCO3 = 0 and pH > 5
#}

#if(which(g[, "HCO3._molality"]== 0 & g[, "CaCO3alkalinity"] > 0 & g[, "pH"] > 5) > 0){
#g <- g[-which(g[, "HCO3._molality"] == 0 & g[, "CaCO3alkalinity"] > 0 & g[, "pH"] > 5), ] # deletes rows above pH 5 where only alkalinity is given.
#}

#if(which(g[,"HCO3._molality"]== 0 & g[,"CaCO3alkalinity"] > 0 & g[,"temp_final"] > 50) > 0){
#g <- g[-which(g[, "HCO3._molality"] == 0 & g[, "CaCO3alkalinity"] > 0 & g[, "temp_final"] > 50), ] # deletes rows where only alkalinity is given and temperature >50. EQ3 can only use alkalinity up to 50 degrees C.
#}


#g[,c("pH", "temp_final", "HCO3._molality", "CaCO3alkalinity")]

#Dissolved gas unit conversion (nanomolality to molality)
g[seq(1, nrow(g), 1),"H2.AQ"] <- as.numeric(g[seq(1, nrow(g), 1), "H2.AQ"])/10^9
g[seq(1, nrow(g), 1),"CO.AQ"] <- as.numeric(g[seq(1, nrow(g), 1), "CO.AQ"])/10^9
g[seq(1, nrow(g), 1),"METHANE.AQ"] <- as.numeric(g[seq(1, nrow(g), 1), "METHANE.AQ"])/10^9

#Ion chromatography unit conversion (ppm to molality)
g[seq(1, nrow(g), 1), "F."] 			<- as.numeric(g[seq(1, nrow(g), 1), "F."])*0.001/18.9984
g[seq(1, nrow(g), 1), "Cl."] 			<- as.numeric(g[seq(1, nrow(g), 1), "Cl."])*0.001/35.453
g[seq(1, nrow(g), 1), "Br."] 			<- as.numeric(g[seq(1, nrow(g), 1), "Br."])*0.001/79.904
g[seq(1, nrow(g), 1), "SO4.2"] 			<- as.numeric(g[seq(1, nrow(g), 1), "SO4.2"])*0.001/96.07
g[seq(1, nrow(g), 1), "PO4.3_final"] 	<- as.numeric(g[seq(1, nrow(g), 1), "PO4.3_final"])*0.001/94.9714
g[seq(1, nrow(g), 1), "NO2._final"] 	<- as.numeric(g[seq(1, nrow(g), 1), "NO2._final"])*0.001/46.0055
g[seq(1, nrow(g), 1), "NO3._final"] 	<- as.numeric(g[seq(1, nrow(g), 1), "NO3._final"])*0.001/62.0049
g[seq(1, nrow(g), 1), "Li."] 			<- as.numeric(g[seq(1, nrow(g), 1), "Li."])*0.001/6.941
g[seq(1, nrow(g), 1), "Na."] 			<- as.numeric(g[seq(1, nrow(g), 1), "Na."])*0.001/22.99
g[seq(1, nrow(g), 1), "NH4._final"] 	<- as.numeric(g[seq(1, nrow(g), 1), "NH4._final"])*0.001/18.01
g[seq(1, nrow(g), 1), "K."] 			<- as.numeric(g[seq(1, nrow(g), 1), "K."])*0.001/39.098
g[seq(1, nrow(g), 1), "Mg.2"] 			<- as.numeric(g[seq(1, nrow(g), 1), "Mg.2"])*0.001/24.305
g[seq(1, nrow(g), 1), "Ca.2"] 			<- as.numeric(g[seq(1, nrow(g), 1), "Ca.2"])*0.001/40.08

#ICP unit conversion (ppb to molality)
g[seq(1, nrow(g), 1), "Li_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Li_ppb"])/6.941/10^6
g[seq(1, nrow(g), 1), "Be_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Be_ppb"])/9.0122/10^6
g[seq(1, nrow(g), 1), "B_ppb"] 		<- as.numeric(g[seq(1, nrow(g), 1), "B_ppb"])/10.811/10^6
g[seq(1, nrow(g), 1), "Mg_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Mg_ppb"])/24.305/10^6
g[seq(1, nrow(g), 1), "Al_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Al_ppb"])/26.9815/10^6
g[seq(1, nrow(g), 1), "Ca_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Ca_ppb"])/40.078/10^6
g[seq(1, nrow(g), 1), "Ti_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Ti_ppb"])/47.88/10^6
g[seq(1, nrow(g), 1), "V_ppb"] 		<- as.numeric(g[seq(1, nrow(g), 1), "V_ppb"])/50.9415/10^6
g[seq(1, nrow(g), 1), "Cr_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Cr_ppb"])/51.9961/10^6
g[seq(1, nrow(g), 1), "Mn_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Mn_ppb"])/54.93805/10^6
g[seq(1, nrow(g), 1), "Fe_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Fe_ppb"])/55.847/10^6
g[seq(1, nrow(g), 1), "Co_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Co_ppb"])/58.9332/10^6
g[seq(1, nrow(g), 1), "Ni_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Ni_ppb"])/58.6934/10^6
g[seq(1, nrow(g), 1), "Cu_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Cu_ppb"])/63.546/10^6
g[seq(1, nrow(g), 1), "Zn_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Zn_ppb"])/65.39/10^6
g[seq(1, nrow(g), 1), "Ga_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Ga_ppb"])/69.723/10^6
g[seq(1, nrow(g), 1), "As_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "As_ppb"])/74.92159/10^6
g[seq(1, nrow(g), 1), "Rb_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Rb_ppb"])/85.4678/10^6
g[seq(1, nrow(g), 1), "Sr_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Sr_ppb"])/87.62/10^6
g[seq(1, nrow(g), 1), "Y_ppb"] 		<- as.numeric(g[seq(1, nrow(g), 1), "Y_ppb"])/88.90585/10^6
g[seq(1, nrow(g), 1), "Zr_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Zr_ppb"])/91.224/10^6
g[seq(1, nrow(g), 1), "Nb_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Nb_ppb"])/92.90638/10^6
g[seq(1, nrow(g), 1), "Mo_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Mo_ppb"])/95.94/10^6
g[seq(1, nrow(g), 1), "Cd_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Cd_ppb"])/112.411/10^6
g[seq(1, nrow(g), 1), "Sn_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Sn_ppb"])/118.71/10^6
g[seq(1, nrow(g), 1), "Sb_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Sb_ppb"])/121.757/10^6
g[seq(1, nrow(g), 1), "Cs_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Cs_ppb"])/132.9054/10^6
g[seq(1, nrow(g), 1), "Ba_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Ba_ppb"])/137.327/10^6
g[seq(1, nrow(g), 1), "La_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "La_ppb"])/138.9055/10^6
g[seq(1, nrow(g), 1), "Hf_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Hf_ppb"])/178.49/10^6
g[seq(1, nrow(g), 1), "W_ppb"] 		<- as.numeric(g[seq(1, nrow(g), 1), "W_ppb"])/183.84/10^6
g[seq(1, nrow(g), 1), "Tl_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Tl_ppb"])/204.3833/10^6
g[seq(1, nrow(g), 1), "Pb_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Pb_ppb"])/207.2/10^6
g[seq(1, nrow(g), 1), "Th_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Th_ppb"])/232.0381/10^6
g[seq(1, nrow(g), 1), "U_ppb"] 		<- as.numeric(g[seq(1, nrow(g), 1), "U_ppb"])/238.0289/10^6
g[seq(1, nrow(g), 1), "Hg_ppb"] 	<- as.numeric(g[seq(1, nrow(g), 1), "Hg_ppb"])/200.59/10^6

# Spectro unit conversion (mg/L or ug/L to molality).
g[seq(1, nrow(g), 1), "O2.AQ"] 				<- as.numeric(g[seq(1, nrow(g), 1), "O2.AQ"])*0.001/31.9988 #convert mg/L to molality
g[seq(1, nrow(g), 1), "Fe.2_spectro"] 		<- as.numeric(g[seq(1, nrow(g), 1), "Fe.2_spectro"])*0.001/55.847 #convert mg/L to molality
g[seq(1, nrow(g), 1), "S.2_spectro_ugL"] 	<- as.numeric(g[seq(1, nrow(g), 1), "S.2_spectro_ugL"])*0.000001/32.066 #convert ug/L to molality
g[seq(1, nrow(g), 1), "SiO2.AQ"] 			<- as.numeric(g[seq(1, nrow(g), 1), "SiO2.AQ"])*0.001/60.086 #convert mg/L to molality
g[seq(1, nrow(g), 1), "NO2._final"] 		<- as.numeric(g[seq(1, nrow(g), 1), "NO2._final"])*0.001/46.01 #convert mg/L to molality
g[seq(1, nrow(g), 1), "NO3._final"] 		<- as.numeric(g[seq(1, nrow(g), 1), "NO3._final"])*0.001/62.00 #convert mg/L to molality
g[seq(1, nrow(g), 1), "PO4.3_final"] 		<- as.numeric(g[seq(1, nrow(g), 1), "PO4.3_final"])*0.001/94.9714 #convert mg/L to molality


### Rename columns to reflect unit change

In [9]:
%%R

### Renaming columns to reflect new units
# Rename the column to molality
colnames(g)[colnames(g) == "Fe.2_spectro"] 		<- "Fe.2_spectro_molality"
colnames(g)[colnames(g) == "S.2_spectro_ugL"] 	<- "S.2_spectro_molality"
colnames(g)[colnames(g) == "SiO2.AQ"] 			<- "SiO2.AQ_spectro_molality"
colnames(g)[colnames(g) == "O2.AQ"] 			<- "O2.AQ_spectro_molality"
colnames(g)[colnames(g) == "H2.AQ"] 			<- "H2.AQ_molality"
colnames(g)[colnames(g) == "METHANE.AQ"] 		<- "METHANE.AQ_molality"
colnames(g)[colnames(g) == "CO.AQ"] 			<- "CO.AQ_molality"
colnames(g)[colnames(g) == "F."] 				<- "F._IC_molality"
colnames(g)[colnames(g) == "Cl."] 				<- "Cl._IC_molality"
colnames(g)[colnames(g) == "Br."] 				<- "Br._IC_molality"
colnames(g)[colnames(g) == "SO4.2"] 			<- "SO4.2_IC_molality"
colnames(g)[colnames(g) == "Li."] 				<- "Li._IC_molality"
colnames(g)[colnames(g) == "Na."] 				<- "Na._IC_molality"
colnames(g)[colnames(g) == "K."] 				<- "K._IC_molality"
colnames(g)[colnames(g) == "Mg.2"] 				<- "Mg.2_IC_molality"
colnames(g)[colnames(g) == "Ca.2"] 				<- "Ca.2_IC_molality"
colnames(g)[colnames(g) == "NH4._final"] 		<- "NH4._final_molality"
colnames(g)[colnames(g) == "NO3._final"] 		<- "NO3._final_molality"
colnames(g)[colnames(g) == "NO2._final"] 		<- "NO2._final_molality"
colnames(g)[colnames(g) == "PO4.3_final"] 		<- "PO4.3_final_molality"

# Rename the column from ppb to molality
colnames(g)[colnames(g) == "Li_ppb"] 		<- "Li_ICP_molality"
colnames(g)[colnames(g) == "Be_ppb"] 		<- "Be_ICP_molality"
colnames(g)[colnames(g) == "B_ppb"] 		<- "B_ICP_molality"
colnames(g)[colnames(g) == "Mg_ppb"] 		<- "Mg_ICP_molality"
colnames(g)[colnames(g) == "Al_ppb"] 		<- "Al_ICP_molality"
colnames(g)[colnames(g) == "Ca_ppb"] 		<- "Ca_ICP_molality"
colnames(g)[colnames(g) == "Ti_ppb"] 		<- "Ti_ICP_molality"
colnames(g)[colnames(g) == "V_ppb"] 		<- "V_ICP_molality"
colnames(g)[colnames(g) == "Cr_ppb"] 		<- "Cr_ICP_molality"
colnames(g)[colnames(g) == "Mn_ppb"] 		<- "Mn_ICP_molality"
colnames(g)[colnames(g) == "Fe_ppb"] 		<- "Fe_ICP_molality"
colnames(g)[colnames(g) == "Co_ppb"] 		<- "Co_ICP_molality"
colnames(g)[colnames(g) == "Ni_ppb"] 		<- "Ni_ICP_molality"
colnames(g)[colnames(g) == "Cu_ppb"] 		<- "Cu_ICP_molality"
colnames(g)[colnames(g) == "Zn_ppb"] 		<- "Zn_ICP_molality"
colnames(g)[colnames(g) == "Ga_ppb"] 		<- "Ga_ICP_molality"
colnames(g)[colnames(g) == "As_ppb"] 		<- "As_ICP_molality"
colnames(g)[colnames(g) == "Rb_ppb"] 		<- "Rb_ICP_molality"
colnames(g)[colnames(g) == "Sr_ppb"] 		<- "Sr_ICP_molality"
colnames(g)[colnames(g) == "Y_ppb"] 		<- "Y_ICP_molality"
colnames(g)[colnames(g) == "Zr_ppb"] 		<- "Zr_ICP_molality"
colnames(g)[colnames(g) == "Nb_ppb"] 		<- "Nb_ICP_molality"
colnames(g)[colnames(g) == "Mo_ppb"] 		<- "Mo_ICP_molality"
colnames(g)[colnames(g) == "Cd_ppb"] 		<- "Cd_ICP_molality"
colnames(g)[colnames(g) == "Sn_ppb"] 		<- "Sn_ICP_molality"
colnames(g)[colnames(g) == "Sb_ppb"] 		<- "Sb_ICP_molality"
colnames(g)[colnames(g) == "Cs_ppb"] 		<- "Cs_ICP_molality"
colnames(g)[colnames(g) == "Ba_ppb"] 		<- "Ba_ICP_molality"
colnames(g)[colnames(g) == "La_ppb"] 		<- "La_ICP_molality"
colnames(g)[colnames(g) == "Hf_ppb"] 		<- "Hf_ICP_molality"
colnames(g)[colnames(g) == "W_ppb"] 		<- "W_ICP_molality"
colnames(g)[colnames(g) == "Tl_ppb"] 		<- "Tl_ICP_molality"
colnames(g)[colnames(g) == "Pb_ppb"] 		<- "Pb_ICP_molality"
colnames(g)[colnames(g) == "Th_ppb"] 		<- "Th_ICP_molality"
colnames(g)[colnames(g) == "U_ppb"] 		<- "U_ICP_molality"
colnames(g)[colnames(g) == "Hg_ppb"] 		<- "Hg_molality"

### Selects measurement to use if multiple measurements are available for a sample, part II
For iron, potassium, calcium, magnesium, and lithium, use ion chromatography if available. Otherwise, use inductively-coupled plasma mass spectrometry (ICP-MS) measurements.


In [10]:
%%R

# Use IC if available. If not, use equivalent ICP values. (Li, Mg, Ca)
g[seq(1, nrow(g), 1), "Li_ICP_molality"] <- ifelse(g[, "Li._IC_molality"] != "",
  g[, "Li._IC_molality"], ifelse(g[, "Li_ICP_molality"]!="", g[, "Li_ICP_molality"], NA))
colnames(g)[colnames(g)=="Li_ICP_molality"] <- "Li_final_molality"
colnames(g)[colnames(g)=="Li._IC_molality"] <- "delete"

g[seq(1, nrow(g), 1), "Mg_ICP_molality"] <- ifelse(g[, "Mg.2_IC_molality"] != "",
  g[, "Mg.2_IC_molality"], ifelse(g[, "Mg_ICP_molality"] != "", g[, "Mg_ICP_molality"], NA))
colnames(g)[colnames(g) == "Mg_ICP_molality"] <- "Mg_final_molality"
colnames(g)[colnames(g) == "Mg.2_IC_molality"] <- "delete"

g[seq(1, nrow(g), 1), "Ca_ICP_molality"] <- ifelse(g[, "Ca.2_IC_molality"] != "",
  g[, "Ca.2_IC_molality"], ifelse(g[, "Ca_ICP_molality"]!="", g[, "Ca_ICP_molality"], NA))
colnames(g)[colnames(g) == "Ca_ICP_molality"] <- "Ca_final_molality"
colnames(g)[colnames(g) == "Ca.2_IC_molality"] <- "delete"

# Use ICP if available. If not, use equivalent spectro values. (Fe)
g[seq(1, nrow(g), 1), "Fe.2_spectro_molality"] <- ifelse(g[, "Fe_ICP_molality"] != "",
  g[, "Fe_ICP_molality"], ifelse(g[, "Fe.2_spectro_molality"] != "", g[, "Fe.2_spectro_molality"], NA))
colnames(g)[colnames(g) == "Fe.2_spectro_molality"] <- "Fe_final_molality"
colnames(g)[colnames(g) == "Fe_ICP_molality"] <- "delete"


### Create a report called 'removed.csv' listing which samples were disregarded/vetted.

In [11]:
%%R

# Compare original file to current matrix to determine which rows have been removed,
# then write a report called "removed.csv" that lists sample names and IDs that didn't make the cut.
f1 <- read.csv(paste(master, ".csv", sep = ""), header=TRUE, colClasses = "character") #re-open original file
a1 <- as.matrix(f1) #convert to matrix and assign to variable a1
# Only consider the first two columns, since additional columns are not comparable
# due to name changes, etc.
a1 <- a1[, c(1, 2)] 
rownames(a1) <- a1[, 1] #assigns rownames
a2 <- g[, c(1, 2)] #variable a2 is the first two columns of the current (modified) matrix, 'g'
rows.in.a1.that.are.not.in.a2  <- function(a1, a2){ #function to compare the two matrices
  a1.vec <- apply(a1, 1, paste, collapse = "")
  a2.vec <- apply(a2, 1, paste, collapse = "")
  a1.without.a2.rows <- a1[!a1.vec %in% a2.vec, ]
  return(a1.without.a2.rows)
}

# Variable 'rowsdeleted' is assigned to the output of the comparison function
rowsdeleted <- rows.in.a1.that.are.not.in.a2(a1, a2)

# Write the 'removed.csv' report, which lists samples that didn't make the cut
write.table(rowsdeleted, file="removed.csv", append=FALSE, row.names=FALSE, col.names=FALSE, sep = ",")

# Delete all columns that contain the word "delete" in the header
g <- g[, -grep(pattern="delete", colnames(g))]

# Check the first five samples to make certain everything is okay so far.
head(g)

        id        name             year   area          pH       temp_final
140114U "140114U" "Sun burn"       "2014" "Falej North" "11.355" "21.7"    
140114V "140114V" "Grey Sauce"     "2014" "Falej North" "11.407" "24.4"    
140111H "140111H" "Horse Head"     "2014" "Qafifah"     "10.24"  "20.2"    
140111I "140111I" "Tadpole Trauma" "2014" "Qafifah"     "10.87"  "18.8"    
140116C "140116C" "Slippity Slide" "2014" "Shmait"      "8.714"  "27.3"    
140116D "140116D" "Rome"           "2014" "Shmait"      "9.104"  "27.1"    
        HCO3._molality         H2.AQ_molality    CO.AQ_molality   
140114U "0.000174717002497918" "1.712594083e-08" "1.62929603e-07" 
140114V "0.000230439916736053" "1.325399218e-06" "2.355398965e-07"
140111H "0.00179496434221482"  "0.000124788739"  "1.85737673e-07" 
140111I "0.000845354662781016" "2.066384969e-06" "9.7319529e-08"  
140116C "0.00474218368026644"  "2.175586009e-06" "1.1363183e-07"  
140116D "0.00397546532889259"  "1.248631188e-05" "2.31419526e-07" 

### Calculate the density of water (rho) and append.

In [12]:
%%R
# The next few calculations require the CHNOSZ package
library(CHNOSZ)
data(thermo)

/Applications/anaconda/envs/py36andR/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: CHNOSZ version 1.1.0 (2017-05-04)

  warnings.warn(x, RRuntimeWarning)
/Applications/anaconda/envs/py36andR/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Please run data(thermo) to create the "thermo" object

  warnings.warn(x, RRuntimeWarning)
/Applications/anaconda/envs/py36andR/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: data(thermo): attached environment "CHNOSZ"

  warnings.warn(x, RRuntimeWarning)
/Applications/anaconda/envs/py36andR/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: thermo$obigt: 1963 aqueous, 3555 total species

  warnings.warn(x, RRuntimeWarning)


In [13]:
%%R
#calculate density of water based on temperature
sample_rho <- water("rho", T = as.numeric(g[, "temp_final"]) + 273.15, P = 1)
sample_rho <- as.character(sample_rho[[1]] / 1000) #convert to g/cm3


In [14]:
%%R
g <- cbind(g, sample_rho) #adds a column named H2O_density and sets it to display calculated rho values
colnames(g)[colnames(g) == 'sample_rho'] <- 'H2O_density'
head(g)

        id        name             year   area          pH       temp_final
140114U "140114U" "Sun burn"       "2014" "Falej North" "11.355" "21.7"    
140114V "140114V" "Grey Sauce"     "2014" "Falej North" "11.407" "24.4"    
140111H "140111H" "Horse Head"     "2014" "Qafifah"     "10.24"  "20.2"    
140111I "140111I" "Tadpole Trauma" "2014" "Qafifah"     "10.87"  "18.8"    
140116C "140116C" "Slippity Slide" "2014" "Shmait"      "8.714"  "27.3"    
140116D "140116D" "Rome"           "2014" "Shmait"      "9.104"  "27.1"    
        HCO3._molality         H2.AQ_molality    CO.AQ_molality   
140114U "0.000174717002497918" "1.712594083e-08" "1.62929603e-07" 
140114V "0.000230439916736053" "1.325399218e-06" "2.355398965e-07"
140111H "0.00179496434221482"  "0.000124788739"  "1.85737673e-07" 
140111I "0.000845354662781016" "2.066384969e-06" "9.7319529e-08"  
140116C "0.00474218368026644"  "2.175586009e-06" "1.1363183e-07"  
140116D "0.00397546532889259"  "1.248631188e-05" "2.31419526e-07" 

### Calculate oxygen fugacity (fO2) and append.

In [15]:
%%R
### Calculate fO2(g) based on temp and O2.AQ
# Uses subcrt to obtain equilibrium constant of O2(aq) = O2(g)
# and multiplies it by [O2.AQ] to obtain fO2(g),then takes the log.
logfO2 <- log10(as.numeric(g[,"O2.AQ_spectro_molality"]) *
  10^subcrt(c("O2", "O2"), c(-1, 1), c("aq", "g"), T = as.numeric(g[, "temp_final"]), P = 1)$out$logK) 
logfO2 <- as.character(logfO2)

#adds a column named fO2 and sets it to display calculated fO2 values
g <- cbind(g, logfO2 = logfO2)
head(g)

/Applications/anaconda/envs/py36andR/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: subcrt: 2 species at 23 values of T and P (wet)

  warnings.warn(x, RRuntimeWarning)


        id        name             year   area          pH       temp_final
140114U "140114U" "Sun burn"       "2014" "Falej North" "11.355" "21.7"    
140114V "140114V" "Grey Sauce"     "2014" "Falej North" "11.407" "24.4"    
140111H "140111H" "Horse Head"     "2014" "Qafifah"     "10.24"  "20.2"    
140111I "140111I" "Tadpole Trauma" "2014" "Qafifah"     "10.87"  "18.8"    
140116C "140116C" "Slippity Slide" "2014" "Shmait"      "8.714"  "27.3"    
140116D "140116D" "Rome"           "2014" "Shmait"      "9.104"  "27.1"    
        HCO3._molality         H2.AQ_molality    CO.AQ_molality   
140114U "0.000174717002497918" "1.712594083e-08" "1.62929603e-07" 
140114V "0.000230439916736053" "1.325399218e-06" "2.355398965e-07"
140111H "0.00179496434221482"  "0.000124788739"  "1.85737673e-07" 
140111I "0.000845354662781016" "2.066384969e-06" "9.7319529e-08"  
140116C "0.00474218368026644"  "2.175586009e-06" "1.1363183e-07"  
140116D "0.00397546532889259"  "1.248631188e-05" "2.31419526e-07" 

### Remove all instances of 'Inf' that result from processing

In [16]:
%%R
g[g == "-Inf"] <- 0 #replace "-Inf" with 0
g[g == "Inf"] <- 0 #replace "Inf" with 0

head(g)

        id        name             year   area          pH       temp_final
140114U "140114U" "Sun burn"       "2014" "Falej North" "11.355" "21.7"    
140114V "140114V" "Grey Sauce"     "2014" "Falej North" "11.407" "24.4"    
140111H "140111H" "Horse Head"     "2014" "Qafifah"     "10.24"  "20.2"    
140111I "140111I" "Tadpole Trauma" "2014" "Qafifah"     "10.87"  "18.8"    
140116C "140116C" "Slippity Slide" "2014" "Shmait"      "8.714"  "27.3"    
140116D "140116D" "Rome"           "2014" "Shmait"      "9.104"  "27.1"    
        HCO3._molality         H2.AQ_molality    CO.AQ_molality   
140114U "0.000174717002497918" "1.712594083e-08" "1.62929603e-07" 
140114V "0.000230439916736053" "1.325399218e-06" "2.355398965e-07"
140111H "0.00179496434221482"  "0.000124788739"  "1.85737673e-07" 
140111I "0.000845354662781016" "2.066384969e-06" "9.7319529e-08"  
140116C "0.00474218368026644"  "2.175586009e-06" "1.1363183e-07"  
140116D "0.00397546532889259"  "1.248631188e-05" "2.31419526e-07" 

### Rearrange columns and create headers

In [17]:
%%R
#rearrange columns
g <- g[, c("id", "name", "year", "area", "temp_final",
  "H2O_density", "logfO2", "pH", "HCO3._molality", "CaCO3alkalinity",
  "O2.AQ_spectro_molality", "S.2_spectro_molality", "SiO2.AQ_spectro_molality", "METHANE.AQ_molality", "H2.AQ_molality",
  "CO.AQ_molality", "F._IC_molality", "Cl._IC_molality", "Br._IC_molality", "SO4.2_IC_molality",
  "PO4.3_final_molality", "NO2._final_molality", "NO3._final_molality", "Li_final_molality", "Na._IC_molality",
  "NH4._final_molality", "K._IC_molality", "Mg_final_molality", "Ca_final_molality", "Be_ICP_molality",
  "B_ICP_molality", "Al_ICP_molality", "Ti_ICP_molality", "V_ICP_molality", "Cr_ICP_molality",
  "Mn_ICP_molality", "Fe_final_molality", "Co_ICP_molality", "Ni_ICP_molality", "Cu_ICP_molality",
  "Zn_ICP_molality", "Ga_ICP_molality", "As_ICP_molality", "Rb_ICP_molality", "Sr_ICP_molality",
  "Y_ICP_molality", "Zr_ICP_molality", "Nb_ICP_molality", "Mo_ICP_molality", "Cd_ICP_molality",
  "Sn_ICP_molality", "Sb_ICP_molality", "Cs_ICP_molality", "Ba_ICP_molality", "La_ICP_molality",
  "Hf_ICP_molality", "W_ICP_molality", "Tl_ICP_molality", "Pb_ICP_molality", "Th_ICP_molality",
  "U_ICP_molality", "Hg_molality")]
head(g)

        id        name             year   area          temp_final
140114U "140114U" "Sun burn"       "2014" "Falej North" "21.7"    
140114V "140114V" "Grey Sauce"     "2014" "Falej North" "24.4"    
140111H "140111H" "Horse Head"     "2014" "Qafifah"     "20.2"    
140111I "140111I" "Tadpole Trauma" "2014" "Qafifah"     "18.8"    
140116C "140116C" "Slippity Slide" "2014" "Shmait"      "27.3"    
140116D "140116D" "Rome"           "2014" "Shmait"      "27.1"    
        H2O_density         logfO2               pH      
140114U "0.99786216466966"  "-0.786192644230544" "11.355"
140114V "0.997214851137051" "-0.825803993360869" "11.407"
140111H "0.998190005466773" "-0.693660329309345" "10.24" 
140111I "0.99847461028675"  "-0.785170218135567" "10.87" 
140116C "0.996441777948425" "-0.727541531960481" "8.714" 
140116D "0.996497585347674" "-0.75336887974908"  "9.104" 
        HCO3._molality         CaCO3alkalinity O2.AQ_spectro_molality
140114U "0.000174717002497918" "17"            "0.00021

### Create header rows
These initially appear in the last two rows and will be moved to the top in the next step.

In [18]:
%%R
# The following creates two rows, one called Headers and one called Units.
# These match the input format for ImbalanceMiner automation
g <- rbind(g,
  Headers = c(
    "id", "name", "year", "area", "Temperature",
    "Density", "log fO2(g)", "H+", "HCO3-", "HCO3-",
    "O2,AQ", "HS-", "SiO2,AQ", "METHANE,AQ", "H2,AQ",
    "CO,AQ", "F-", "Cl-", "Br-", "SO4-2",
    "PO4-3", "NO2-", "NO3-", "Li+", "Na+",
    "NH4+", "K+", "Mg+2", "Ca+2", "Be+2",
    "B(OH)3,AQ", "Al+3", "Ti+4", "VO+2", "CrO4-2",
    "Mn+2", "Fe+2", "Co+2", "Ni+2", "Cu+2",
    "Zn+2", "Ga+3", "H2AsO4-", "Rb+", "Sr+2",
    "Y+3", "Zr+4", "NbO3-", "MoO4-2", "Cd+2",
    "Sn+2", "SbO2-", "Cs+", "Ba+2", "La+3",
    "Hf+4", "WO4-2", "Tl+", "Pb+2", "Th+4",
    "U+4", "Hg+2"),
  Units = c(
    "id", "name", "year", "area", "tempC",
    "g/cm3", "fo2lgi", "pH", "Molality", "Alk., mg/L CaCO3",
    rep("Molality", 52)))
tail(g)

        id        name            year   area     temp_final   
140117G "140117G" "Lisas Revenge" "2014" "Shmait" "30.5"       
140117H "140117H" "Sheep Warbler" "2014" "Shmait" "29.6"       
140117K "140117K" "Lizard Tamer"  "2014" "Shmait" "27.4"       
140117L "140117L" "Dead Cricket"  "2014" "Shmait" "27.3"       
Headers "id"      "name"          "year" "area"   "Temperature"
Units   "id"      "name"          "year" "area"   "tempC"      
        H2O_density         logfO2              pH       HCO3._molality        
140117G "0.995500601023217" "-2.07037918808829" "11.389" "1.85475861781848e-05"
140117H "0.995774345556474" "-1.73065676830124" "11.485" "2.87840363030808e-05"
140117K "0.996413738546434" "-1.51100877187498" "11.491" "3.19959751040799e-05"
140117L "0.996441777948425" "-1.74576635206668" "11.561" "1.58025716069942e-05"
Headers "Density"           "log fO2(g)"        "H+"     "HCO3-"               
Units   "g/cm3"             "fo2lgi"            "pH"     "Molality"     

### Move the two header rows to the top

In [19]:
%%R

g <- g[c(nrow(g) -1, nrow(g), seq(1, nrow(g)-2), 1), ]
head(g)

        id        name             year   area          temp_final   
Headers "id"      "name"           "year" "area"        "Temperature"
Units   "id"      "name"           "year" "area"        "tempC"      
140114U "140114U" "Sun burn"       "2014" "Falej North" "21.7"       
140114V "140114V" "Grey Sauce"     "2014" "Falej North" "24.4"       
140111H "140111H" "Horse Head"     "2014" "Qafifah"     "20.2"       
140111I "140111I" "Tadpole Trauma" "2014" "Qafifah"     "18.8"       
        H2O_density         logfO2               pH      
Headers "Density"           "log fO2(g)"         "H+"    
Units   "g/cm3"             "fo2lgi"             "pH"    
140114U "0.99786216466966"  "-0.786192644230544" "11.355"
140114V "0.997214851137051" "-0.825803993360869" "11.407"
140111H "0.998190005466773" "-0.693660329309345" "10.24" 
140111I "0.99847461028675"  "-0.785170218135567" "10.87" 
        HCO3._molality         CaCO3alkalinity    O2.AQ_spectro_molality
Headers "HCO3-"                

### Final preprocessing cleanup

In [20]:
%%R

g[is.na(g)] <- "" #replace NAs with ""

g[g == ""] <- 0 #replace "" with 0

#for all concentrations (currently column 9 and beyond) that are reported as negative, replace with '0'
for (i in 3:length(g[, 1])){
  for (ii in 9:length(g[1, ])){
    if (g[i, ii] <= 0){
      g[i, ii] <- 0
    }
  }
}

head(g)

        id        name             year   area          temp_final   
Headers "id"      "name"           "year" "area"        "Temperature"
Units   "id"      "name"           "year" "area"        "tempC"      
140114U "140114U" "Sun burn"       "2014" "Falej North" "21.7"       
140114V "140114V" "Grey Sauce"     "2014" "Falej North" "24.4"       
140111H "140111H" "Horse Head"     "2014" "Qafifah"     "20.2"       
140111I "140111I" "Tadpole Trauma" "2014" "Qafifah"     "18.8"       
        H2O_density         logfO2               pH      
Headers "Density"           "log fO2(g)"         "H+"    
Units   "g/cm3"             "fo2lgi"             "pH"    
140114U "0.99786216466966"  "-0.786192644230544" "11.355"
140114V "0.997214851137051" "-0.825803993360869" "11.407"
140111H "0.998190005466773" "-0.693660329309345" "10.24" 
140111I "0.99847461028675"  "-0.785170218135567" "10.87" 
        HCO3._molality         CaCO3alkalinity    O2.AQ_spectro_molality
Headers "HCO3-"                

### Create a cleaned-up version of the master input file
This file is called 'MasterInput_cleaned.csv' and will be processed into EQ3 input files.

In [21]:
%%R
paste("default working directory:", getwd())
wd <- getwd()
paste("directory where modified input .csv will be generated:", wd)
setwd(wd)
write.table(g, file="MasterInput_cleaned.csv", append=FALSE, row.names=FALSE, col.names=FALSE, sep=",")

# Create EQ3 input files
### First, open 'MasterInput_cleaned.csv' and create the rxn_3i folder that will be used to store .3i input files

In [22]:
%%R
# cleaned input file should be in the format:

# row 1: names of the variables that will be input into the file, with correct EQ3 species names (e.g. SO4--) 
# row 2: units/constraint of the variables that are recognizable in the EQ3 file (e.g. Suppressed, mg/L, Molarity)
# row 3+: values for the sample sites (e.g. Bison OF1, Bison OF2)

# col 1: sample IDs
# col 2: sample names
# col 3: sample year
# col 4: sample area
# col 5: temperature, C
# col 6: water density at the given temperature
# col 7: logfO2 for the given temperature, [O2,AQ]
# col 8: pH
# col 9: HCO3- molality from DIC
# col 10: CaCO3 alkalinity
# col 11+: basis species and measurements for input

f <- read.csv("MasterInput_cleaned.csv", header=FALSE, colClasses = "character")
g <- as.matrix(f)

In [23]:
%%R
# delete the rxn_3i directory if it exists
unlink("rxn_3i", recursive = TRUE)

In [24]:
%%R
# create a fresh rxn_3i directory
dir.create("rxn_3i", showWarnings = FALSE)

In [25]:
%%R
paste("default working directory:", getwd())
wd <- getwd()
input_path <- paste(wd, "/rxn_3i", sep="")
paste("directory where 3i input files will be generated:", input_path)
setwd(input_path)

### For each sample, fill out an 'EQ3 input' text template

In [26]:
%%R
for (ii in 3:length(g[, 1])){

  eq3.filename <- paste(c(ii-2, ".3i"), collapse = "") #create the name of the file from row number-2 (e.g. "1.3i")
  eq3.filename <- gsub(" ", "", eq3.filename) #remove an unecessary space

  eq3.header <- "|------------------------------------------------------------------------------|
| Title                  | (utitl(n))                                          |
|------------------------------------------------------------------------------|
|                                                                              |
|"
  eq3.samplename <- paste(c("Sample:", format(g[ii, 1], width=70)), collapse = " ")

  eq3.header2 <- "|
|                                                                              |
|------------------------------------------------------------------------------|
|Special Basis Switches (for model definition only)       | (nsbswt)           |
|------------------------------------------------------------------------------|
|Replace |None                                            | (usbsw(1,n))       |
|   with |None                                            | (usbsw(2,n))       |
|------------------------------------------------------------------------------|
|Temperature (C)         | "

  eq3.temperature <- sprintf("%.5E", as.numeric(g[ii, 5]))

  eq3.headeroptions1 <- "| (tempc)                                |
|------------------------------------------------------------------------------|
|Pressure option (jpres3):                                                     |
|  [ ] ( 0) Data file reference curve value                                    |
|  [ ] ( 1) 1.013-bar/steam-saturation curve value                             |
|  [x] ( 2) Value (bars) | 1.00000E+00| (press)                                |
|------------------------------------------------------------------------------|
|Density (g/cm3)         | "

  eq3.density <- sprintf("%.5E", as.numeric(g[ii, 6]))

  eq3.headeroptions2 <- "| (rho)                                  |
|------------------------------------------------------------------------------|
|Total dissolved solutes option (itdsf3):                                      |
|  [x] ( 0) Value (mg/kg.sol) | 0.00000E+00| (tdspkg)                          |
|  [ ] ( 1) Value (mg/L)      | 0.00000E+00| (tdspl)                           |
|------------------------------------------------------------------------------|
|Electrical balancing option (iebal3):                                         |
|  [x] ( 0) No balancing is done                                               |
|  [ ] ( 1) Balance on species |None                    | (uebal)              |
|------------------------------------------------------------------------------|
|Default redox constraint (irdxc3):                                            |
|  [ ] (-3) Use O2(g) line in the aqueous basis species block                  |
|  [ ] (-2) pe (pe units)      | 0.00000E+00| (pei)                            |
|  [ ] (-1) Eh (volts)         | 5.00000E-01| (ehi)                            |
|  [x] ( 0) Log fO2 (log bars) | "

  if (as.numeric(g[ii, 7]) != 0 && g[ii, 7] != ""){
    eq3.O2fugacity <- sprintf("%.4E", as.numeric(g[ii, 7]))
  } else if (g[ii, 7] == "Inf" || g[ii, 7]== "-Inf"){
    eq3.O2fugacity <- sprintf("%.4E", -6)
  } else {
    eq3.O2fugacity <- sprintf("%.4E", -6)
  }

  eq3.headeroptions3 <- "| (fo2lgi)                         |
|  [ ] ( 1) Couple (aux. sp.)  |None                    | (uredox)             |
|------------------------------------------------------------------------------|
|Aqueous Basis Species/Constraint Species        |Conc., etc. |Units/Constraint|
| (uspeci(n)/ucospi(n))                          | (covali(n))|(ujf3(jflgi(n)))|
|------------------------------------------------------------------------------|"

  writeme1 <- paste(c(eq3.header, eq3.samplename, eq3.header2, eq3.temperature, eq3.headeroptions1,
  eq3.density, eq3.headeroptions2, eq3.O2fugacity, eq3.headeroptions3), collapse = "")
  write(writeme1, file=eq3.filename, append = FALSE)
  
  eq3.pHtitle	<- format("H+", width = 48)
  eq3.pHvalue	<- sprintf("%.5E", as.numeric(g[ii, 8]))
  eq3.pHflag	<- format("pH", width = 16)
  
  writemepH <- paste(c("|", eq3.pHtitle, "| ", eq3.pHvalue, "|", eq3.pHflag, "|"), collapse="")
  write(writemepH, file=eq3.filename,append=TRUE)
  
  if (as.numeric(g[ii, 9]) > 0){
    eq3.carbonvalue <- g[ii, 9]
    eq3.carbonflag <- "Molality"
  }
  
  if (as.numeric(g[ii, 9]) == 0 && as.numeric(g[ii, 10]) > 0){
  eq3.carbonvalue <- g[ii, 10]
    eq3.carbonflag <- "Alk., mg/L CaCO3"
  }
  
  if (as.numeric(g[ii, 9]) == 0 && as.numeric(g[ii, 10]) == 0){
    eq3.carbonvalue <- 0
    eq3.carbonflag <- "Molality"
  }
  
  eq3.carbontitle <- format("HCO3-", width = 48)
  eq3.carbonvalue <- sprintf("%.5E", as.numeric(eq3.carbonvalue))
  eq3.carbonflag <- format(eq3.carbonflag, width = 16)
  
  writemecarbon <- paste(c("|", eq3.carbontitle, "| ", eq3.carbonvalue,
    "|", eq3.carbonflag, "|"), collapse="")
  write(writemecarbon, file=eq3.filename, append=TRUE)

  for (i in 11:length(g[1, ])){
  
    eq3.base1title <- format((g)[1, i], width = 48)
    eq3.base1title
  
    eq3.base1value <- sprintf("%.5E", as.numeric(g[ii, i]))
    eq3.base1value
  
    eq3.base1flag <- format((g)[2, i], width = 16)
    eq3.base1flag
  
    writeme2 <- paste(c("|", eq3.base1title, "| ", eq3.base1value, "|", eq3.base1flag, "|"), collapse="")
    write(writeme2, file=eq3.filename, append = TRUE)
  
  } # End loop

  eq3.ender <- "|N2,AQ                                           | 0.00000E+00|Suppressed      |
|CN-                                             | 0.00000E+00|Suppressed      |
|------------------------------------------------------------------------------|
* Valid jflag strings (ujf3(jflgi(n))) are:                                    *
*    Suppressed          Molality            Molarity                          *
*    mg/L                mg/kg.sol           Alk., eq/kg.H2O                   *
*    Alk., eq/L          Alk., eq/kg.sol     Alk., mg/L CaCO3                  *
*    Alk., mg/L HCO3-    Log activity        Log act combo                     *
*    Log mean act        pX                  pH                                *
*    pHCl                pmH                 pmX                               *
*    Hetero. equil.      Homo. equil.        Make non-basis                    *
*------------------------------------------------------------------------------*
|Create Ion Exchangers  | (net)                                                |
|------------------------------------------------------------------------------|
|Advisory: no exchanger creation blocks follow on this file.                   |
|Option: on further processing (writing a PICKUP file or running XCON3 on the  |
|present file), force the inclusion of at least one such block (qgexsh):       |
|  [ ] (.true.)                                                                |
|------------------------------------------------------------------------------|
|Ion Exchanger Compositions      | (neti)                                      |
|------------------------------------------------------------------------------|
|Exchanger phase |None                    | (ugexpi(n))                        |
|------------------------------------------------------------------------------|
|->|Moles/kg.H2O    |  0.0000    | (cgexpi(n))                                 |
|------------------------------------------------------------------------------|
|->|Exchange site   |None    | (ugexji(j,n))                                   |
|------------------------------------------------------------------------------|
|--->|Exchange species        |Eq. frac.   | (this is a table header)          |
|------------------------------------------------------------------------------|
|--->|None                    | 0.00000E+00| (ugexsi(i,j,n), egexsi(i,j,n))    |
|------------------------------------------------------------------------------|
|Solid Solution Compositions     | (nxti)                                      |
|------------------------------------------------------------------------------|
|Solid Solution          |None                    | (usoli(n))                 |
|------------------------------------------------------------------------------|
|->|Component               |Mole frac.  | (this is a table header)            |
|------------------------------------------------------------------------------|
|->|None                    | 0.00000E+00| (umemi(i,n), xbari(i,n))            |
|------------------------------------------------------------------------------|
|Alter/Suppress Options  | (nxmod)                                             |
|------------------------------------------------------------------------------|
|Species                                         |Option          |Alter value |
| (uxmod(n))                                     |(ukxm(kxmod(n)))| (xlkmod(n))|
|------------------------------------------------------------------------------|
|None                                            |None            | 0.00000E+00|
|------------------------------------------------------------------------------|
* Valid alter/suppress strings (ukxm(kxmod(n))) are:                           *
*    Suppress            Replace             AugmentLogK                       *
*    AugmentG                                                                  *
*------------------------------------------------------------------------------*
|Iopt Model Option Switches (\"( 0)\" marks default choices)                     |
|------------------------------------------------------------------------------|
|iopt(4) - Solid Solutions:                                                    |
|  [x] ( 0) Ignore                                                             |
|  [ ] ( 1) Permit                                                             |
|------------------------------------------------------------------------------|
|iopt(11) - Auto Basis Switching in pre-N-R Optimization:                      |
|  [x] ( 0) Turn off                                                           |
|  [ ] ( 1) Turn on                                                            |
|------------------------------------------------------------------------------|
|iopt(17) - PICKUP File Options:                                               |
|  [ ] (-1) Don't write a PICKUP file                                          |
|  [x] ( 0) Write a PICKUP file                                                |
|------------------------------------------------------------------------------|
|iopt(19) - Advanced EQ3NR PICKUP File Options:                                |
|  [x] ( 0) Write a normal EQ3NR PICKUP file                                   |
|  [ ] ( 1) Write an EQ6 INPUT file with Quartz dissolving, relative rate law  |
|  [ ] ( 2) Write an EQ6 INPUT file with Albite dissolving, TST rate law       |
|  [ ] ( 3) Write an EQ6 INPUT file with Fluid 1 set up for fluid mixing       |
|------------------------------------------------------------------------------|
|Iopg Activity Coefficient Option Switches (\"( 0)\" marks default choices)      |
|------------------------------------------------------------------------------|
|iopg(1) - Aqueous Species Activity Coefficient Model:                         |
|  [ ] (-1) The Davies equation                                                |
|  [x] ( 0) The B-dot equation                                                 |
|  [ ] ( 1) Pitzer's equations                                                 |
|  [ ] ( 2) HC + DH equations                                                  |
|------------------------------------------------------------------------------|
|iopg(2) - Choice of pH Scale (Rescales Activity Coefficients):                |
|  [ ] (-1) \"Internal\" pH scale (no rescaling)                                 |
|  [x] ( 0) NBS pH scale (uses the Bates-Guggenheim equation)                  |
|  [ ] ( 1) Mesmer pH scale (numerically, pH = -log m(H+))                     |
|------------------------------------------------------------------------------|
|Iopr Print Option Switches (\"( 0)\" marks default choices)                     |
|------------------------------------------------------------------------------|
|iopr(1) - Print All Species Read from the Data File:                          |
|  [x] ( 0) Don't print                                                        |
|  [ ] ( 1) Print                                                              |
|------------------------------------------------------------------------------|
|iopr(2) - Print All Reactions:                                                |
|  [x] ( 0) Don't print                                                        |
|  [ ] ( 1) Print the reactions                                                |
|  [ ] ( 2) Print the reactions and log K values                               |
|  [ ] ( 3) Print the reactions, log K values, and associated data             |
|------------------------------------------------------------------------------|
|iopr(3) - Print the Aqueous Species Hard Core Diameters:                      |
|  [x] ( 0) Don't print                                                        |
|  [ ] ( 1) Print                                                              |
|------------------------------------------------------------------------------|
|iopr(4) - Print a Table of Aqueous Species Concentrations, Activities, etc.:  |
|  [ ] (-3) Omit species with molalities < 1.e-8                               |
|  [ ] (-2) Omit species with molalities < 1.e-12                              |
|  [ ] (-1) Omit species with molalities < 1.e-20                              |
|  [x] ( 0) Omit species with molalities < 1.e-100                             |
|  [ ] ( 1) Include all species                                                |
|------------------------------------------------------------------------------|
|iopr(5) - Print a Table of Aqueous Species/H+ Activity Ratios:                |
|  [x] ( 0) Don't print                                                        |
|  [ ] ( 1) Print cation/H+ activity ratios only                               |
|  [ ] ( 2) Print cation/H+ and anion/H+ activity ratios                       |
|  [ ] ( 3) Print ion/H+ activity ratios and neutral species activities        |
|------------------------------------------------------------------------------|
|iopr(6) - Print a Table of Aqueous Mass Balance Percentages:                  |
|  [ ] (-1) Don't print                                                        |
|  [x] ( 0) Print those species comprising at least 99% of each mass balance   |
|  [ ] ( 1) Print all contributing species                                     |
|------------------------------------------------------------------------------|
|iopr(7) - Print Tables of Saturation Indices and Affinities:                  |
|  [ ] (-1) Don't print                                                        |
|  [x] ( 0) Print, omitting those phases undersaturated by more than 10 kcal   |
|  [ ] ( 1) Print for all phases                                               |
|------------------------------------------------------------------------------|
|iopr(8) - Print a Table of Fugacities:                                        |
|  [ ] (-1) Don't print                                                        |
|  [x] ( 0) Print                                                              |
|------------------------------------------------------------------------------|
|iopr(9) - Print a Table of Mean Molal Activity Coefficients:                  |
|  [x] ( 0) Don't print                                                        |
|  [ ] ( 1) Print                                                              |
|------------------------------------------------------------------------------|
|iopr(10) - Print a Tabulation of the Pitzer Interaction Coefficients:         |
|  [x] ( 0) Don't print                                                        |
|  [ ] ( 1) Print a summary tabulation                                         |
|  [ ] ( 2) Print a more detailed tabulation                                   |
|------------------------------------------------------------------------------|
|iopr(17) - PICKUP file format (\"W\" or \"D\"):                                   |
|  [x] ( 0) Use the format of the INPUT file                                   |
|  [ ] ( 1) Use \"W\" format                                                     |
|  [ ] ( 2) Use \"D\" format                                                     |
|------------------------------------------------------------------------------|
|Iodb Debugging Print Option Switches (\"( 0)\" marks default choices)           |
|------------------------------------------------------------------------------|
|iodb(1) - Print General Diagnostic Messages:                                  |
|  [x] ( 0) Don't print                                                        |
|  [ ] ( 1) Print Level 1 diagnostic messages                                  |
|  [ ] ( 2) Print Level 1 and Level 2 diagnostic messages                      |
|------------------------------------------------------------------------------|
|iodb(3) - Print Pre-Newton-Raphson Optimization Information:                  |
|  [x] ( 0) Don't print                                                        |
|  [ ] ( 1) Print summary information                                          |
|  [ ] ( 2) Print detailed information (including the beta and del vectors)    |
|  [ ] ( 3) Print more detailed information (including matrix equations)       |
|  [ ] ( 4) Print most detailed information (including activity coefficients)  |
|------------------------------------------------------------------------------|
|iodb(4) - Print Newton-Raphson Iteration Information:                         |
|  [x] ( 0) Don't print                                                        |
|  [ ] ( 1) Print summary information                                          |
|  [ ] ( 2) Print detailed information (including the beta and del vectors)    |
|  [ ] ( 3) Print more detailed information (including the Jacobian)           |
|  [ ] ( 4) Print most detailed information (including activity coefficients)  |
|------------------------------------------------------------------------------|
|iodb(6) - Print Details of Hypothetical Affinity Calculations:                |
|  [x] ( 0) Don't print                                                        |
|  [ ] ( 1) Print summary information                                          |
|  [ ] ( 2) Print detailed information                                         |
|------------------------------------------------------------------------------|
|Numerical Parameters                                                          |
|------------------------------------------------------------------------------|
| Beta convergence tolerance      | 0.00000E+00| (tolbt)                       |
| Del convergence tolerance       | 0.00000E+00| (toldl)                       |
| Max. Number of N-R Iterations   |   0        | (itermx)                      |
|------------------------------------------------------------------------------|
|Ordinary Basis Switches (for numerical purposes only)    | (nobswt)           |
|------------------------------------------------------------------------------|
|Replace |None                                            | (uobsw(1,n))       |
|   with |None                                            | (uobsw(2,n))       |
|------------------------------------------------------------------------------|
|Sat. flag tolerance     | 0.00000E+00| (tolspf)                               |
|------------------------------------------------------------------------------|
|Aq. Phase Scale Factor  | 1.00000E+00| (scamas)                               |
|------------------------------------------------------------------------------|
|End of problem                                                                |
|------------------------------------------------------------------------------|"

write(eq3.ender, file=eq3.filename, append = TRUE)

} # End loop

### Create two additional .3i files for charge-balance on pH (...pH.3i) and bicarbonate (...c.3i)

In [27]:
%%R
# This section of the code takes the generated .3i files, sets them to
# balance on pH, and saves a copy with a filename = file number + "pH.3i"
quantity3i <- length(list.files())

for (i in 1:quantity3i){

  FileRead <- readLines(paste(c(toString(i), ".3i"), collapse=""))
  FileRead <- gsub("[[:punct:]]x[[:punct:]] [[:punct:]] 0[[:punct:]] No balancing is done", "[ ] ( 0) No balancing is done", FileRead)
  FileRead <- gsub("[[:punct:]] [[:punct:]] [[:punct:]] 1[[:punct:]] Balance on species [[:punct:]]None                    [[:punct:]] [[:punct:]]uebal[[:punct:]]", "[x] ( 1) Balance on species |H+                      | (uebal)", FileRead)

  write(FileRead, paste(c(toString(i), "pH.3i"), collapse = ""))

} #End loop

#this section of the code takes the generated .3i files, sets them to balance on HCO3-, and saves a copy with a filename = file number +"carb.3i"
for (i in 1:quantity3i){

  FileRead <- readLines(paste(c(toString(i), ".3i"), collapse = ""))
  FileRead <- gsub("[[:punct:]]x[[:punct:]] [[:punct:]] 0[[:punct:]] No balancing is done", "[ ] ( 0) No balancing is done", FileRead)
  FileRead <- gsub("[[:punct:]] [[:punct:]] [[:punct:]] 1[[:punct:]] Balance on species [[:punct:]]None                    [[:punct:]] [[:punct:]]uebal[[:punct:]]", "[x] ( 1) Balance on species |HCO3-                   | (uebal)", FileRead)

  write(FileRead, paste(c(toString(i), "c.3i"), collapse = ""))

} # End loop

### Go back to parent directory

In [28]:
%%R
getwd()
setwd(wd)
getwd()

[1] "/Users/graysonboyer/Charge Balance and Aqueous Speciation"


# Run EQ3 for each .3i file

### Define necessary python functions

In [29]:
def read_inputs(file_type, location):
    ### this function can find all .6o files in all downstream folders, hence the additio of file_list
    file_name = [] # file names
    file_list = [] # file names with paths
    for root, dirs, files in os.walk(location):
        for file in files:
            if file.endswith(file_type):
                file_name.append(file)
                file_list.append(os.path.join(root, file))
    return file_name, file_list


def mk_check_del_directory(path):
    ###  This code checks for the dir being created, and if it is already present, deletes it, before recreating it
    if not os.path.exists(path):           #    Check if the dir is alrady pessent
        os.makedirs(path)                  #    Build desired output directory
    else:
        shutil.rmtree(path)                #    Remove directory and contents if it is already present.
        os.makedirs(path)
def call_EQ3(l, path, water):
    ### Calls and runs EQ3 in a serial fashion.
    ### l = data0 3 leter suffix of a data0/1 file in th db directory
    ### path = path to where ever the 3i files are stored
    ### water = the name of the 3i file itself
    
    print('calling EQ3 on ' + water + ' using ' + l)
    os.chdir(path)                                              #    step into 3i folder 
    args = ['/bin/csh', '/Users/tuckerely/SUPCRTandEQs/Automate_EQ3/bin/runeq3', l, water]        #    Arguements to be run inside csh. ./runeq3 data0_suffix .3i file
    with open(os.devnull, 'w') as fp:                           #    use of devnull will allow me to supress written output, and hopefull speed things up.
        Popen(args, stdout=fp).wait()                           #    wait should give time for process to complete 
#     sleep(0.5)                                                  #    this is added because wait doesnt always seem to work, it is a temproary fix to keep runs from overlapping

    os.chdir(pwd)                                               #    move back to parent directory     
    
    try:
        ### rename output
        shutil.move('rxn_3i/output', 'rxn_3o/' + water[:-1] + 'o')   #     local calling of outut should work given code execution location
#         shutil.move('rxn_3i/pickup', 'rxn_3p/' + water[:-1] + 'p')

    
    ### If any of the outputs are not correct, notify of failure.
    except:
        print('EQ3 failed to produce output on ' + water)
        sys.exit()


### Run .3i files

In [30]:
mk_check_del_directory('rxn_3o')
mk_check_del_directory('rxn_3p')
three_i_files, three_i_file_paths = read_inputs('3i', 'rxn_3i')

input_dir = pwd + "/rxn_3i/"
pickup_dir = pwd + "/rxn_3p/"

for i in three_i_files:
    call_EQ3('jus', 'rxn_3i', i)
    
    # rename and move pickup files if generated
    os.chdir(input_dir)
    i_trunc = re.split('(?<=[a-zA-Z0-9_]).3i$', i)[0]
    try:
        os.rename("pickup", i_trunc + ".3p")
        shutil.move(input_dir + i_trunc + ".3p", pickup_dir + i_trunc + ".3p")
    except:
        print("pickup file for " + i + " not generated")
    
    os.chdir(pwd)

calling EQ3 on 1.3i using jus
calling EQ3 on 10.3i using jus
calling EQ3 on 10c.3i using jus
calling EQ3 on 10pH.3i using jus
calling EQ3 on 11.3i using jus
calling EQ3 on 11c.3i using jus
calling EQ3 on 11pH.3i using jus
calling EQ3 on 12.3i using jus
calling EQ3 on 12c.3i using jus
calling EQ3 on 12pH.3i using jus
calling EQ3 on 13.3i using jus
calling EQ3 on 13c.3i using jus
calling EQ3 on 13pH.3i using jus
calling EQ3 on 14.3i using jus
calling EQ3 on 14c.3i using jus
calling EQ3 on 14pH.3i using jus
calling EQ3 on 15.3i using jus
calling EQ3 on 15c.3i using jus
calling EQ3 on 15pH.3i using jus
calling EQ3 on 16.3i using jus
calling EQ3 on 16c.3i using jus
calling EQ3 on 16pH.3i using jus
calling EQ3 on 17.3i using jus
calling EQ3 on 17c.3i using jus
calling EQ3 on 17pH.3i using jus
calling EQ3 on 18.3i using jus
calling EQ3 on 18c.3i using jus
calling EQ3 on 18pH.3i using jus
calling EQ3 on 19.3i using jus
calling EQ3 on 19c.3i using jus
calling EQ3 on 19pH.3i using jus
calling EQ

# Postprocessing of .3o files and generate a report

### Change directory to rxn_3o folder where .3o files have been generated

In [31]:
%%R
getwd()
wd <- getwd()
output_path <- paste(wd, "/rxn_3o", sep="")
setwd(output_path)
getwd()

[1] "/Users/graysonboyer/Charge Balance and Aqueous Speciation/rxn_3o"


### Read each .3o file and extract charge balance, ionic strength, and Eh.
Then generate 'report.csv'.

In [32]:
%%R

# There should be a .3o, c.3o and pH.3o for each sample, thus divide by 3
numberofsamples <- length(list.files(pattern = "\\.3o$"))/3

for (i in 1:numberofsamples){
    
  setwd(output_path)
  
  FileRead <- toString(readLines(paste(c(toString(i), ".3o"), collapse = "")))

  filenumber <- i
   
  match <- regexec("Sample:\\s*\\w+", FileRead)
  if (match[[1]][1] != -1){
    Sample_Name <- regmatches(FileRead, match)
    Sample_Name <- tail(strsplit(Sample_Name[[1]], split=" ")[[1]], 1)
  } else {
    Sample_Name <- -999
  }

  match <- regexec("(\\S+)\\s*per cent of the total charge", FileRead)
  if (match[[1]][1] != -1){
    Percent_Imbalance <- regmatches(FileRead, match)
    Percent_Imbalance <- head(strsplit(Percent_Imbalance[[1]], split = " ")[[1]], 1)
  } else {
    Percent_Imbalance <- -999
  }

  match <- regexec("NBS pH scale                \\S+     \\S+", FileRead)
  if (match[[1]][1] != -1){
    Eh_calc <- regmatches(FileRead, match)
    Eh_calc <- tail(strsplit(Eh_calc[[1]],split=" ")[[1]],1)
  } else {
    Eh_calc <- -999
  }

  match <- regexec("Ionic strength [[:punct:]]I[[:punct:]]= \\S+", FileRead)
  if (match[[1]][1] != -1){
    IS_calc <- regmatches(FileRead, match)
    IS_calc <- tail(strsplit(IS_calc[[1]], split = " ")[[1]], 1)
  } else {
    IS_calc <- -999
  }

  FileRead2 <- toString(readLines(paste(c(toString(i), "ph.3o"),collapse = "")))
  match <- regexec("Final[[:blank:]]+[[:alnum:]]+[[:punct:]][[:alnum:]]+", FileRead2)
  if (match[[1]][1] != -1){
    pH_balanced_calc <- regmatches(FileRead2, match)
    pH_balanced_calc <- tail(strsplit(pH_balanced_calc[[1]], split=" ")[[1]], 1)
    pH_balanced_calc <- substr(pH_balanced_calc, 1, nchar(pH_balanced_calc)-1)
  } else {
    pH_balanced_calc <- -999
  }

  FileRead3 <- toString(readLines(paste(c(toString(i), "c.3o"), collapse = "")))
  match <- regexec("Final[[:blank:]]+[[:alnum:]]+[[:punct:]][[:alnum:]]+", FileRead3)
  match2 <- regexec("[[:alnum:]]+[[:punct:]][[:alnum:]]+[[:blank:]]+mg/L HCO3-", FileRead3)

  if (match[[1]][1] != -1){
    carb_balanced_calc <- regmatches(FileRead3, match)
    carb_balanced_calc <- tail(strsplit(carb_balanced_calc[[1]], split = " ")[[1]], 1)
    carb_balanced_calc <- substr(carb_balanced_calc, 1, nchar(carb_balanced_calc)-1)
  } else if (match2[[1]][1] != -1){
    carb_balanced_calc <- regmatches(FileRead3, match2)
    carb_balanced_calc <- head(strsplit(carb_balanced_calc[[1]], split = " ")[[1]], 1)
    carb_balanced_calc <- substr(carb_balanced_calc, 1, nchar(carb_balanced_calc)-1)
  } else {
    carb_balanced_calc <- -999
  }

  setwd(wd)

  g <- as.matrix(read.csv("MasterInput_cleaned.csv", header=FALSE, colClasses = "character"))
  colnames(g) <- g[1, ]
  colnames(g)[9] <- "HCO3._molality"
  colnames(g)[10] <- "CaCO3alkalinity"
  geochem <- g[i+2, seq(2, length(g[i, ]), 1)]

  ProtonErrorPercent <- 100*(((10^-(as.numeric(pH_balanced_calc))) -
    (10^-(as.numeric(g[i+2, "H+"]))))/(10^-(as.numeric(g[i+2, "H+"]))))

  carbinputDIC <- as.numeric(g[i+2, "HCO3._molality"])*1000*61.0168
  carbinputALK <- as.numeric(g[i+2, "CaCO3alkalinity"])*1.22 #convert CaCO3 mg/L alkalinity to HCO3- mg/L alkalinity

  if (carbinputDIC > 0){
    carbinput <- carbinputDIC
    carbmessage <- "DIC"
  }

  if (carbinputDIC == 0 && carbinputALK > 0){
    carbinput <- carbinputALK
    carbmessage <- "CaCO3"
  }

  if (carbinputDIC == 0 && carbinputALK == 0){
    carbinput <- 0
    carbmessage <- "No DIC or CaCO3"
  }

  DIC_Error <- as.numeric(carb_balanced_calc) - carbinput

  number_of_columns <- 12 + length(g[i, ]) - 1 # 1) file 2) sample 3) pH 4)charge-balanced pH 5) proton error percent 6) %imbalance 7) Eh 8) IS 9+) geochem's column length -1 (redundant geochem sample column removed)

  headers1 <- data.frame(matrix(c("", "", "", "", "HCO3- input",
    "Charge-balanced HCO3-", "Method", "Charge Imbalance", "[H+] discrepancy", "HCO3- discrepancy",
    "Eh", "IS", g[1, seq(2, length(g[1, ]), 1)]), nrow = 1, ncol = number_of_columns))
  headers2 <- data.frame(matrix(c("File", "Sample", "pH", "Charge-balanced pH", "mg/L",
    "mg/L", "Type", "%", "%", "mg/L",
    "volts", "mol/kg", g[2, seq(2, length(g[1, ]), 1)]), nrow=1, ncol=number_of_columns))
  sample_entry <- data.frame(matrix(c(filenumber, Sample_Name, g[i+2,"H+"], pH_balanced_calc,
    carbinput, carb_balanced_calc, carbmessage, Percent_Imbalance, ProtonErrorPercent,
    DIC_Error, Eh_calc, IS_calc, geochem), nrow = 1, ncol = number_of_columns))

  setwd(wd)
    if (i==1){
      write.table(headers1, file = "report.csv", sep = ",",
        append=FALSE, row.names = FALSE, col.names=FALSE)
      write.table(headers2, file = "report.csv", sep = ",",
        append=TRUE, row.names = FALSE, col.names=FALSE)
      write.table(sample_entry, file = "report.csv", sep = ",",
        append=TRUE, row.names = FALSE, col.names=FALSE)
    } else {
      write.table(sample_entry, file = "report.csv", sep = ",",
        append=TRUE, row.names = FALSE, col.names=FALSE)
    }
} # End loop

print("Done!")

[1] "Done!"
